<a href="https://colab.research.google.com/github/yu072333/114-/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 新增區段

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Dk1HCgX0yPg3gHSbxZL8bNssT5PaR5NvIRW3K7fd9M4/edit?usp=sharing')

In [ ]:
!pip install gradio -q
import gradio as gr
import pandas as pd
from datetime import datetime

# 記帳資料存放在 '工作表1' 工作表
SHEET_NAME = '工作表1'

# 定義品項和支付方式選項
CATEGORIES = ['外食', '買菜', '交通', '購物', '娛樂', '其他']
PAYMENTS = ['電子支付', '信用卡', '現金']




In [ ]:
# 核心函式 1: 寫入新記錄 (Input to GoogleSheet)
def add_new_expense(date, category, amount, payment_method, is_essential):
    """
    將一筆新的消費記錄寫入 Google 試算表。
    (滿足結構要求: if/else, def)
    """
    try:
        worksheet = gsheets.worksheet(SHEET_NAME)
    except Exception as e:
        return f"❌ 錯誤：無法找到或連接工作表 '{SHEET_NAME}'。請檢查工作表名稱和權限。"
       # **使用 if/else 判斷並標準化 '是否必需' 的值**
    if is_essential:
        essential_status = '是'
    else:
        essential_status = '否'
     # 準備寫入的數據列
    new_row = [date, category, amount, payment_method, essential_status]

    # 寫入到工作表的最後一行
    worksheet.append_row(new_row)

    return f"✅ 新增記錄成功：{date}，{category}，${amount} 元。"

In [ ]:
#核心函式 2: 分析與寫回 Google Sheet (Output from GoogleSheet & Input to GoogleSheet)
def analyze_and_update():
    """
    從 Google Sheet 讀取數據、分析消費習慣，並將分析結果寫入另一個工作表。
    (滿足結構要求: for, def)
    """
    # --- 讀取數據 (Output from GoogleSheet) ---
    worksheet = gsheets.worksheet(SHEET_NAME)
    data = worksheet.get_all_values()

    if len(data) <= 1:
        return pd.DataFrame(), "❌ 試算表中沒有足夠的消費數據可供分析。", "寫入狀態：無數據，未寫入。"

    # 轉換為 DataFrame，假設欄位順序固定
    df = pd.DataFrame(data[1:], columns=data[0])
    df.columns = ['日期', '品項', '金額', '支付方式', '是否必需']
    df['金額'] = pd.to_numeric(df['金額'], errors='coerce').fillna(0)

    # --- 數據分析 (使用 for 迴圈示範) ---
    total_spending = df['金額'].sum()
    essential_spending = 0
    non_essential_spending = 0

    # **使用 for 迴圈遍歷，並用 if 判斷**
    # 統計必需與非必需消費
    for index, row in df.iterrows():
        if row['是否必需'] == '是':
            essential_spending += row['金額']
        else:
            non_essential_spending += row['金額']

    # 按品項彙總
    category_summary = df.groupby('品項')['金額'].sum().sort_values(ascending=False).reset_index()
    category_summary.rename(columns={'金額': '總花費金額 (元)'}, inplace=True)

    # 創建分析結果 DataFrame
    analysis_df = pd.DataFrame({
        '分析項目': ['總消費', '必需消費', '非必需消費', '必需品佔比'],
        '金額或數值': [
            total_spending,
            essential_spending,
            non_essential_spending,
            f"{essential_spending/total_spending*100:.2f}%" if total_spending > 0 else "0%"
        ]
    })

    # --- 寫回 Google Sheet (Input to GoogleSheet) ---

    # 將品項彙總和分析結果寫入 '分析報告' 工作表
    try:
        output_worksheet = gsheets.worksheet('分析報告')
    except gspread.WorksheetNotFound:
        output_worksheet = gsheets.add_worksheet(title="分析報告", rows="100", cols="20")

    output_worksheet.clear()

    # 寫入品項彙總
    output_worksheet.update('A1', [category_summary.columns.values.tolist()] + category_summary.values.tolist())

    # 寫入空白行區隔
    output_worksheet.update('A' + str(len(category_summary) + 3), [['消費習慣分析']])

    # 寫入分析結果
    output_worksheet.update('A' + str(len(category_summary) + 4), [analysis_df.columns.values.tolist()] + analysis_df.values.tolist())

    summary_text = (
        f"**總消費金額：** ${total_spending:,.2f} 元\n"
        f"**必需花費佔比：** {analysis_df.loc[3, '金額或數值']}"
    )
    write_status = "✅ 分析結果已成功寫入 Google 試算表中的 **分析報告** 工作表！"

    return category_summary, summary_text, write_status

In [ ]:
# --- Gradio 介面配置與啟動 ---

# A. 記帳輸入介面 (Input)
input_tab = gr.Interface(
    fn=add_new_expense,
    inputs=[
        gr.Textbox(label="日期 (例如: 2023/10/26)", value=datetime.now().strftime("%Y/%m/%d")),
        gr.Dropdown(CATEGORIES, label="品項/類別", value='外食'),
        gr.Number(label="金額", minimum=0),
        gr.Radio(PAYMENTS, label="支付方式", value='電子支付'),
        gr.Checkbox(label="是否必需 (勾選為 '是')", value=True)
    ],
    outputs=gr.Markdown(label="寫入狀態"),
    title="新增消費記錄",
    description="在此填寫您的消費資料，資料將直接寫入 Google Sheet 的 '記帳資料' 工作表。"
)

# B. 數據分析介面 (Output)
output_tab = gr.Interface(
    fn=analyze_and_update,
    inputs=None, # 分析不需要額外輸入
    outputs=[
        gr.Dataframe(label="各品項消費彙總"),
        gr.Markdown(label="主要分析摘要"),
        gr.Markdown(label="Google Sheet 寫入狀態")
    ],
    title="消費數據分析與報告",
    description="點擊 Submit 讀取最新數據，進行分析，並將報告寫入 '分析報告' 工作表。",
    live=False # 不實時更新，需按 Submit
)

# 組合兩個介面到一個 Gradio Tabs
accountant_app = gr.TabbedInterface(
    [input_tab, output_tab],
    ["🧾 記帳輸入", "📊 數據分析與報告"],
    title="家庭記帳與消費習慣分析工具"
)

# 啟動 Gradio 介面
accountant_app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a6e886d7a25683898.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
